In [ ]:
import os
import pyarrow as pa
import pyarrow.parquet as parquet
from astropy.io import fits
import pandas as pd
import sys 
sys.path.insert(0, '/home/bekah/gPhoton2')
from gPhoton.io.mast import download_data
import matplotlib.pyplot as plt 

In [ ]:
fuv = pd.read_csv("/home/bekah/glcat/masks/every3_fuv.csv")

eclipses = fuv.iloc[::20]

In [ ]:
header_variables = ['FDTTDC', 'ITFUVOBA', 'FDHVMON', 'FDHIMON', 'FDSFHV', 'FDCTEC', 'FDCFEC', 'TRANGE0', 'TRANGE1']
#header_variables = ['NDTTDC', 'ITNUVOBA', 'NDHVMON', 'NDHIMON', 'NDSFHV', 'NDCTEC', 'NDCFEC', 'TRANGE0', 'TRANGE1']


In [ ]:

data = []

for index, eclipse in eclipses['eclipse'].items():
    try:
        file_path = download_data(eclipse, 'scst')
        with fits.open(file_path) as hdul:
            header = hdul[0].header
            row_data = {var: header.get(var, None) for var in header_variables}
            row_data['eclipse'] = eclipse 
            data.append(row_data)
            
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        
    finally:
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Deleted {file_path}")
            
df = pd.DataFrame(data)
output_parquet = 'nuv_scst_data.parquet'
table = pa.Table.from_pandas(df)
pq.write_table(table, output_parquet)
print(f"Data saved to {output_parquet}")

In [ ]:
fuv_scst = parquet.read_table("/home/bekah/glcat/notebooks/background/fuv_scst_data.parquet").to_pandas()

In [ ]:
fuv_scst['eclipse'] = fuv_scst['eclipse'].astype(int)

def convert_to_date(trange_str):
    year = int(trange_str[:2]) + 2000  
    month = int(trange_str[2:4])
    return pd.to_datetime(f"{year}-{month}-01")

fuv_scst['TRANGE0'] = fuv_scst['TRANGE0'].apply(convert_to_date)
fuv_scst['TRANGE1'] = fuv_scst['TRANGE1'].apply(convert_to_date)

In [ ]:
n_vars = len(header_variables)
n_cols = 2
n_rows = (n_vars + 1) // n_cols 

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, n_rows * 3))

axes = axes.flatten()

for i, var in enumerate(header_variables):
    axes[i].scatter(fuv_scst['TRANGE0'], fuv_scst[var], s=0.5)
    axes[i].set_xlabel('TRANGE0')
    axes[i].set_ylabel(var)
    axes[i].set_title(f'{var} vs Date')

# Remove any empty subplots (if odd number of variables)
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()

plt.show()

In [ ]:
fuv_scst